In [ ]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Load and preprocess a grayscale image
def preprocess_image(image_path):
    # Read an image in grayscale mode
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

# Extract centroid and grayscale value for each cell
def extract_features(image):
    # Find external contours in the input image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    features = []
    for contour in contours:
        # Calculate moments of the contour
        M = cv2.moments(contour)
        if M["m00"] == 0:
            continue
        # Calculate the centroid coordinates (cX, cY)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # Calculate the mean grayscale value at the centroid
        mean_value = np.mean(image[cY, cX])
        features.append([cX, cY, mean_value])
    return features

# Perform clustering using K-Means and get optimized cluster centers
def find_optimized_centers(train_features, num_clusters):
    # Create a K-Means object with the specified number of clusters and n_init to suppress the warning
    kmeans = KMeans(n_clusters=num_clusters, n_init=10)
    # Fit the K-Means model on the training features to find cluster centers
    kmeans.fit(train_features)
    return kmeans.cluster_centers_

# Segment test images using cluster centers
def segment_test_images(test_images, cluster_centers):
    labels = []
    for test_image in test_images:
        # Extract features from the test image
        test_features = extract_features(test_image)
        # Calculate the Euclidean distances from test features to cluster centers
        distances = np.linalg.norm(np.array(test_features)[:, :2] - cluster_centers[:, :2], axis=1)
        # Find the closest cluster for each test feature
        closest_cluster = np.argmin(distances)
        labels.append(closest_cluster)
    return labels

if __name__ == "__main__":
    # Directory paths for training and test images
    train_dir = "./cell_input/temp"
    test_dir = "./cell_input/Test1/"

    # Process multiple training images
    training_image_paths = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]
    all_features = []
    for image_path in training_image_paths:
        # Preprocess the training image (load it in grayscale mode)
        image = preprocess_image(image_path)
        # Extract features from the training image
        image_features = extract_features(image)
        # Collect all features from the training images
        all_features.extend(image_features)

    # Define the number of clusters (optimized centers)
    num_clusters = 3  # Adjust as needed
    # Find optimized cluster centers based on the training data
    optimized_centers = find_optimized_centers(np.array(all_features), num_clusters)
    print(optimized_centers)

    # Load and preprocess test images
    test_image_paths = [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)]
    test_images = [preprocess_image(image_path) for image_path in test_image_paths]

    # Segment test images using cluster centers
    test_labels = segment_test_images(test_images, optimized_centers)

    # Visualize the segmented test images
    for i, label in enumerate(np.unique(test_labels)):
        plt.figure()
        plt.imshow(test_images[i], cmap='viridis')  # Set the colormap to 'viridis'
        plt.title(f"Segmented as Cluster {label + 1}")
        plt.show()

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import os

# Function to extract 2-dimensional feature vectors from image patches
def extract_feature_vectors(image, patch_size=7, shift=1):
    feature_vectors = []
    height, width = image.shape

    for y in range(0, height - patch_size + 1, shift):
        for x in range(0, width - patch_size + 1, shift):
            patch = image[y:y + patch_size, x:x + patch_size]
            mean = np.mean(patch)
            variance = np.var(patch)
            feature_vectors.append([mean, variance])

    return feature_vectors

# Function to apply K-Means clustering to feature vectors and get optimized cluster centers
def find_optimized_centers(feature_vectors, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(feature_vectors)
    cluster_centers = kmeans.cluster_centers_
    return cluster_centers

# Function to segment test images using cluster centers
def segment_test_images(test_images, cluster_centers):
    labels = []
    for test_image in test_images:
        test_feature_vectors = extract_feature_vectors(test_image)
        test_labels = []
        for feature_vector in test_feature_vectors:
            distances = np.linalg.norm(np.array(feature_vector) - cluster_centers, axis=1)
            closest_cluster = np.argmin(distances)
            test_labels.append(closest_cluster)
        labels.append(test_labels)
    return labels

# Load and preprocess an image
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

# Directory paths for training and test images
train_dir = "./cell_input/temp/"
test_dir = "./cell_input/Test1/"

# Define the number of clusters (optimized centers)
num_clusters = 3  # Adjust as needed

# Process training images
training_feature_vectors = []
for filename in os.listdir(train_dir):
    image = preprocess_image(os.path.join(train_dir, filename))
    feature_vectors = extract_feature_vectors(image)
    training_feature_vectors.extend(feature_vectors)

# Find optimized cluster centers from training feature vectors
optimized_centers = find_optimized_centers(training_feature_vectors, num_clusters)

# Process test images and segment them using the cluster centers
test_images = []
for filename in os.listdir(test_dir):
    test_image = preprocess_image(os.path.join(test_dir, filename))
    test_images.append(test_image)

test_labels = segment_test_images(test_images, optimized_centers)

# Display the segmented test images
for i, test_image in enumerate(test_images):
    segmented_image = np.zeros_like(test_image)
    for label in test_labels[i]:
        patch_index = np.where(np.array(test_labels[i]) == label)[0]  # Find patches with the current label
        for idx in patch_index:
            y, x = idx // (test_image.shape[1] - 7), idx % (test_image.shape[1] - 7)
            patch = test_image[y:y + 7, x:x + 7]
            segmented_image[y:y + 7, x:x + 7] = 255

    cv2.imshow(f"Segmented Image - {os.listdir(test_dir)[i]}", segmented_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()